In [1]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from scipy.misc import derivative
from pathlib import Path
from matplotlib.offsetbox import AnchoredText

%matplotlib inline

In [2]:
#Make a dataframe from the sample directory. Good to visualize data. You should 
#feed this dataframe to another function that will make an array of objects

def gen_spec_df(folder_dir, meta_dic):
    
    #initialize a df of the CSV sample/scan info.
    #This should probably be something I iron out with path (it'd be a '\' on PC) but it's fine for now
    raw_df = pd.read_csv(folder_dir + "/" + meta_dic)
    
    #get the hdf filename for every file in the directory
    files = []
    for x in os.listdir(folder_dir):
        if x.endswith(".hdf5"):
            files.append(x)
    
    #Add a column for filename in meta_df that is the same length of the meta_df
    edxname = [''] * len(raw_df.index)
    raw_df['FileName'] = edxname
    
    #Find each file in the meta_df by comparing scan number. Add file name 
    for y in range(len(files)):
        hdf_file = h5py.File(directory + files[y], 'r')
        for z in range(len(raw_df.index)):
            if raw_df['ScanNum'].values[z] == hdf_file.attrs['Scan Number']:
                raw_df['FileName'].values[z] = directory+files[y]
    
    #Add a column for EDX data arrays
    edxd = [''] * len(raw_df.index)
    raw_df['Spectra'] = edxd
    raw_df['Position'] = edxd
    
    #Scan through filenames and add position and spectra data arrays to df
    #If there's none, set those values equal to some string or warning
    for h in range(len(raw_df.index)):
        datahdf = h5py.File(raw_df['FileName'].values[h], 'r')
        positionflag = 'No Position Data (uh oh)'
        spectraflag = 'No Spectra Data (uh oh)'
        with h5py.File(raw_df['FileName'].values[h], 'r') as f:
            for x in range(len(f.keys())):
                if list(f.keys())[x] == '7bmb1:aero:m2.VAL':
                    raw_df['Position'].values[h] = f['7bmb1:aero:m2.VAL'][...]
                    positionflag = 'yep'
                if list(f.keys())[x] == 'dxpMercury:mca1.VAL':
                    raw_df['Spectra'].values[h] = f['dxpMercury:mca1.VAL'][...]
                    spectraflag = 'yep'
        if positionflag != 'yep':
            raw_df['Position'].values[h] = []
        if spectraflag != 'yep':
            raw_df['Spectra'].values[h] = []

    return raw_df

In [3]:
class scan:
    
    normalization = 1
    cal_feat_1 = (275, 2292.1)
    cal_feat_2 = (934, 8397.6)
    units = 'bins'
    
    def __init__(self, spectra, position_from_center):
        self.spectra = spectra
        self.beam_position = position_from_center

    def set_energy_scale(self, tuple1, tuple2):
        self.cal_feat_1 = tuple1
        self.cal_feat_2 = tuple2
        
    def set_units(self, unit):
        if unit == 'energy' or unit == 'bins' or unit == '2 theta':
            self.units = unit
        else: print('Unit must be set to "energy" , "2 theta", or "bins"')

    def get_energy_scale(self): return (self.cal_feat_1, self.cal_feat_2)

    def get_units(self): return (self.units)
         
    def quick_plot(self):
        spec_data = self.spectra
        x_dat = list(range(len(spec_data)))
        fig, ax = plt.subplots()
        ax.plot(x_dat, spec_data, linewidth=2.0)
        ax.grid(True);
        return (fig, ax)
    
    def quick_scatter(self):
        spec_data = self.spectra
        x_dat = list(range(len(spec_data)))
        fig, ax = plt.subplots()
        ax.scatter(x_dat, spec_data)
        ax.grid(True);
        return (fig, ax)
    
    def quick_scatter1(self):
        spec_data = self.spectra
        x_dat = list(range(len(spec_data)))
        x_dat_en = []
        x1 = self.cal_feat_1[0]
        y1 = self.cal_feat_1[1]
        x2 = self.cal_feat_2[0]
        y2 = self.cal_feat_2[1]
        for a in range(len(x_dat)):
            lin_interp = (y2-y1)*(a-x1)/(x2-x1) + y1
            x_dat_en.append(lin_interp)
        fig, ax = plt.subplots(figsize = (10,6))
        ax.plot(x_dat_en, spec_data)
        ax.grid(True);
        return (fig, ax)
    
    def bins_to_energy(self, x_data_array):
        x_data_en = []
        x1 = self.cal_feat_1[0]
        y1 = self.cal_feat_1[1]
        x2 = self.cal_feat_2[0]
        y2 = self.cal_feat_2[1]
        for a in range(len(x_data_array)):
            lin_interp = (y2-y1)*(x_data_array[a]-x1)/(x2-x1) + y1
            x_data_en.append(lin_interp)
        return x_data_en

    def bins_to_energy2(self):
        spec_data = self.spectra
        x_data = list(range(len(spec_data)))
        x_dat_en = []
        x1 = self.cal_feat_1[0]
        y1 = self.cal_feat_1[1]
        x2 = self.cal_feat_2[0]
        y2 = self.cal_feat_2[1]
        for a in range(len(x_data)):
            lin_interp = (y2-y1)*(a-x1)/(x2-x1) + y1
            x_dat_en.append(lin_interp)
        return x_data_en
        

In [4]:
#When bool_pos_given_as_index is True, the pos instantiation variable is an index. Otherwise, when false,
#the pos instantiation variable is an actual position value, like -13.0 or 6.75. It also catches
#an edge case when the position array is empty, such as the calibration scan 1069. Things may behave funny when
#position arrays end in duplicates of 0s or any duplicates in general

#There's potentially a problem for if the spectra passed to the scan is a list or a numpy array, be aware.
def scan_instant_helper(df, scannum, pos, bool_pos_given_as_index):
    errorflag = True
    pos_i = None
    scan_index = (df[df['ScanNum']==scannum].index.values)[0]
    pos_array = df['Position'][scan_index]
    
    #edge case checker, if the position array of a given scan number is empty
    if ((pos == 0) and len(pos_array)==0):
        spectra_set = df['Spectra'][scan_index]
        spectra = spectra_set
        return scan(spectra, None)

    #If the bool is given as true, treat the pos variable like an index and not a value
    #Check first that the pos as an index is within the range of the scan's position array
    if bool_pos_given_as_index == True:
        if pos > (len(pos_array)-1): 
            return print('ERROR: Index is out of range of position array')
        else:
            pos_i = pos
            pos_value = abs(df['Position'][scan_index][pos_i])
            spectra_set = df['Spectra'][scan_index]
            spectra = spectra_set[pos_i]
            return scan(spectra, pos_value)
    
    #Otherwise, the bool was false and the pos variable was given as an actual position in the array
    #Check first that the pos value is actually in the array, and if so instantiate the scan with appropriate info
    else:
        for a in range(len(pos_array)):
            pos_val = pos_array[a]
            #This old logic in the 'or' statement was to catch the empty edge case, it should be unnecessary now
            if abs(pos_array[a])==abs(pos) or ((pos == 0) and len(pos_array)==0): 
                errorflag = False
                pos_i = a
                
        if errorflag == True: print('ERROR: Position is out of range')
        else:
            spectra_set = df['Spectra'][scan_index]
            spectra = spectra_set[pos_i]
            position_val = abs(df['Position'][scan_index][pos_i])
            return scan(spectra, position_val)

In [5]:
class sample:
    
    recipe_ph = None
    recipe_steel = None
    
    aging_temp = None
    aging_atm = None
    aging_humidity = None
    aging_time = None
    
    def __init__(self, sample_name, scan_set): #, file_location):
        self.name = sample_name
        self.scans = scan_set
        self.scan_len = len(self.scans)
        
    #Return an array of the positions of each scan in the scan list
    def get_positions(self):
        length = self.scan_len
        scan_array = self.scans
        scan_pos = []
        for a in range(length):
            if (scan_array[a]).beam_position == None:
                scan_pos.append(0)
            else:
                scan_pos.append((scan_array[a]).beam_position)
        return scan_pos
            
    #Set attributes of the class, the instantiation helper function should do most of this
    def set_ph(self, hi_low_string): self.recipe_ph = hi_low_string
    def set_steel_fiber(self, true_false): self.recipe_steel = true_false
    def set_temp(self, temperature): self.aging_temp = temperature
    def set_atm(self, atmosphere): self.aging_atm = atmosphere
    def set_humidity(self, rh): self.aging_humidity = rh
    def set_time(self, days): self.aging_time = days


    #Return values of each attribute
    def get_ph(self): return (self.recipe_ph)
    def get_steel_fiber(self): return (self.recipe_steel)    
    def get_temp(self): return (self.aging_temp)    
    def get_atm(self): return (self.aging_atm)
    def get_humidity(self): return (self.aging_humidity)
    def get_time(self): return (self.aging_time)
        
    def quick_plot(self):
        return
    
    def quick_stack(self):
        return
        
    def plot(self):
        return
    
    def stackplot(self):
        return
    
    def plot_d(self):
        return
    
    def stackplot_d(self):
        return

In [6]:
def sample_instant_helper(df, scannum, name):
    errorflag = True
    scan_obj_list = []
    scan_index = (df[df['ScanNum']==scannum].index.values)
    scan_pos_arr = df['Position'][scan_index]
    
#     #Edge check, if pos array is empty
#     if len(scan_pos_arr)==0:
#         spectra_set = df['Spectra'][scan_index]
#         spectra = spectra_set
#         scan_obj_list.append(scan(spectra, None))
    
#     #Otherwise make a scan object at each position and add scan object to a list
#     else:
    for a in range(len(scan_pos_arr)):
        spectra_set = df['Spectra'][scan_index]
        spectra = spectra_set[a]
        pos = scan_pos_arr[a]
        scan_obj_list.append(scan(spectra, pos))
    
    #make the sample object with the proper list of scans
    sample_obj = sample(str(name), scan_obj_list)
    
    #set attributes of the sample object from values in the data frame
    # pH = df['pH'][scan_index].values
    # steel = df['SteelFiber'][scan_index].item()
    # temp = df['Temperature'][scan_index].item()
    # atm = df['Atmosphere'][scan_index].item()
    # #humidity = df['Humidity'][scan_index].item()
    # days = df['Days'][scan_index].item()
    
#     if pH[0] == 1:
#         pH = 'High'
#     elif pH[0] == 0:
#         pH = 'Low'
#     else:
#         pH = None
        
    # if steel == True:
    #         steel = True
    # elif steel == 0:
    #      steel == False

        
    # sample_obj.set_ph(pH)
    # # sample_obj.set_steel_fiber(steel)
    # # sample_obj.set_temp(temp)
    # # sample_obj.set_atm(str(atm))
    # # #sample_obj.set_humidity(rh)
    # # sample_obj.set_time(str((days)) + ' Days')
    
    return scan_obj_list



In [9]:
#sample_df['Temperature'][20]
Low_SS_60C_6d = sample_instant_helper(sample_df, 'Low pH SS 60C 6 Day 100% CO2', 1086)


#type(Low_SS_60C_6d)

Low_SS_60C_6d

#Low_SS_60C_6d.get_positions()
#print(Low_SS_60C_6d.aging_temp.values)


# print(Low_SS_60C_6d.get_steel_fiber())
# print(Low_SS_60C_6d.get_temp())
# print(Low_SS_60C_6d.get_atm())
# print(Low_SS_60C_6d.get_humidity())
# print(Low_SS_60C_6d.get_time())

#type(float(sample_df['Temperature'][20]))

#type(sample_df['Temperature'][20].item())


[]

In [11]:
scan_n = 1106

s1 = scan_instant_helper(sample_df, scan_n, 13, False)
s2 = scan_instant_helper(sample_df, scan_n, 12.75, False)
s3 = scan_instant_helper(sample_df, scan_n, 12.5, False)
s4 = scan_instant_helper(sample_df, scan_n, 12.25, False)
s5 = scan_instant_helper(sample_df, scan_n, 12, False)
s6 = scan_instant_helper(sample_df, scan_n, 11.75, False)
s7 = scan_instant_helper(sample_df, scan_n, 11.5, False)
s8 = scan_instant_helper(sample_df, scan_n, 11.25, False)
s9 = scan_instant_helper(sample_df, scan_n, 11, False)
s10 = scan_instant_helper(sample_df, scan_n, 10.75, False)
s11 = scan_instant_helper(sample_df, scan_n, 10.5, False)
s12 = scan_instant_helper(sample_df, scan_n, 10.25, False)
s13 = scan_instant_helper(sample_df, scan_n, 10, False)
s14 = scan_instant_helper(sample_df, scan_n, 9.75, False)
s15 = scan_instant_helper(sample_df, scan_n, 9.5, False)
s16 = scan_instant_helper(sample_df, scan_n, 9.25, False)
s17 = scan_instant_helper(sample_df, scan_n, 9, False)
s18 = scan_instant_helper(sample_df, scan_n, 8.75, False)
s19 = scan_instant_helper(sample_df, scan_n, 8.5, False)
s20 = scan_instant_helper(sample_df, scan_n, 8.25, False)
s21 = scan_instant_helper(sample_df, scan_n, 8, False)
s22 = scan_instant_helper(sample_df, scan_n, 7.75, False)
s23 = scan_instant_helper(sample_df, scan_n, 7.5, False)
s24 = scan_instant_helper(sample_df, scan_n, 7.25, False)
s25 = scan_instant_helper(sample_df, scan_n, 7, False)
s26 = scan_instant_helper(sample_df, scan_n, 6.75, False)
s27 = scan_instant_helper(sample_df, scan_n, 6.5, False)
s28 = scan_instant_helper(sample_df, scan_n, 6.25, False)
s29 = scan_instant_helper(sample_df, scan_n, 6, False)

slist = [s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26,s27,s28,s29]

hpbl2514 = sample('High pH BL 25C 14d', slist)
hpbl2514.set_ph('High')
hpbl2514.set_steel_fiber(False)
hpbl2514.set_temp(25)
hpbl2514.set_atm('100% CO2')
hpbl2514.set_time(14)

In [12]:
print(hpbl2514.get_ph())
print(hpbl2514.get_steel_fiber())
print(hpbl2514.get_temp())
print(hpbl2514.get_atm())
print(hpbl2514.get_time())

print('\n')

print(type(hpbl2514.get_ph()))
print(type(hpbl2514.get_steel_fiber()))
print(type(hpbl2514.get_temp()))
print(type(hpbl2514.get_atm()))
print(type(hpbl2514.get_time()))

print('\n')

print(hpbl2514.get_positions())
print(hpbl2514.scan_len)

High
False
25
100% CO2
14


<class 'str'>
<class 'bool'>
<class 'int'>
<class 'str'>
<class 'int'>


[13.0, 12.75, 12.5, 12.25, 12.0, 11.75, 11.5, 11.25, 11.0, 10.75, 10.5, 10.25, 10.0, 9.75, 9.5, 9.25, 9.0, 8.75, 8.5, 8.25, 8.0, 7.75, 7.5, 7.25, 7.0, 6.75, 6.5, 6.25, 6.0]
29


In [11]:
sample_df

,ScanNum,FullName,pH,Days,Temperature,Atmosphere,SteelFiber,Aborted,Notes,Start (mm),End (mm),Step Size (mm),Slit Entrance (um),Scan Time,Rotation Speed (deg/s),FileName,Spectra,Position
0,1066,Foil Scan 1,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,NaN,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 221.0, 295.0, 314.0, 334.0, 354.0, 354....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1067,Foil Scan 2,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,NaN,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 203.0, 278.0, 317.0, 330.0, 369.0, 349....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1068,Foil Scan 3,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,NaN,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 849.0, 945.0, 1174.0, 1282.0, 1385.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1069,Mo W Energy Calibration,NaN,NaN,NaN,NaN,NaN,False,EDX Calibration,NaN,NaN,NaN,NaN,90.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 5366.0, 6316.0, 7253.0, 7844.0, 8473.0, ...",[]
4,1070,LaB6,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,90.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 5138.0, 6247.0, 7093.0, 7730.0, 8211.0, ...",[]
5,1071,Calcite (CaCO3),NaN,NaN,NaN,NaN,NaN,False,EDX Calibration,NaN,NaN,NaN,NaN,90.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 5150.0, 6166.0, 7062.0, 7600.0, 8218.0, ...",[]
6,1072,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 1054.0, 1230.0, 1444.0, 1533.0, 1597.0, ...",[]
7,1073,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 106.0, 120.0, 128.0, 174.0, 164.0, 185....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1074,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 98.0, 110.0, 135.0, 170.0, 159.0, 162.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1075,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 6217.0, 7384.0, 8390.0, 9519.0, 9998.0, ...",[]


In [144]:
def sample_instant_helper(df, scannum, name):
    errorflag = True
    scan_obj_list = []
    scan_index = (df[df['ScanNum']==scannum].index.values)
    scan_pos_arr = df['Position'][scan_index]
    
    #Edge check, if pos array is empty
    if len(scan_pos_arr)==0:
        spectra_set = df['Spectra'][scan_index]
        spectra = spectra_set
        scan_obj_list.append(scan(spectra, None))
    
    #Otherwise make a scan object at each position and add scan object to a list
    else:
        for a in range(len(scan_pos_arr)):
            spectra_set = df['Spectra'][scan_index]
            spectra = spectra_set[a]
            pos = scan_pos_arr[a]
            scan_obj_list.append(scan(spectra, pos))
    
    #make the sample object with the proper list of scans
    sample_obj = sample(str(name), scan_obj_list)
    
    #set attributes of the sample object from values in the data frame
    # pH = df['pH'][scan_index].values
    # steel = df['SteelFiber'][scan_index].item()
    # temp = df['Temperature'][scan_index].item()
    # atm = df['Atmosphere'][scan_index].item()
    # #humidity = df['Humidity'][scan_index].item()
    # days = df['Days'][scan_index].item()
    
#     if pH[0] == 1:
#         pH = 'High'
#     elif pH[0] == 0:
#         pH = 'Low'
#     else:
#         pH = None
        
    # if steel == True:
    #         steel = True
    # elif steel == 0:
    #      steel == False

        
    # sample_obj.set_ph(pH)
    # # sample_obj.set_steel_fiber(steel)
    # # sample_obj.set_temp(temp)
    # # sample_obj.set_atm(str(atm))
    # # #sample_obj.set_humidity(rh)
    # # sample_obj.set_time(str((days)) + ' Days')
    
    return scan_obj_list



In [ ]:
Mo_W = scan_instant_helper(sample_df, 1069, 0, False)
calcite = scan_instant_helper(sample_df, 1071, 0, True)

t = sample('Please work you stupid bastard', [Mo_W, calcite])

print(t.name)


In [ ]:
low_ss = sample('Low pH with Steel', [0,1,2])
low = sample('Low pH', [3,4,5,6])
hi_ss = sample('High pH with Steel', [6])
hi = sample('High pH', [9,0])

low_ss.set_ph('Low')
low.set_ph('Low')
hi_ss.set_ph('High')
hi.set_ph('High')

print(low_ss.get_ph())
print(low.get_ph())
print(hi_ss.get_ph())
print(hi.get_ph())

print('\n')

print(low_ss.recipe_pH)
print(low.recipe_pH)
print(hi_ss.recipe_pH)
print(hi.recipe_pH)

In [ ]:
low_ss.scan_list()
#low.get_num_of_scans()

In [ ]:
print(low_ss.get_num_of_scans())
print(low.get_num_of_scans())
print(hi_ss.get_num_of_scans())
print(hi.get_num_of_scans())

In [ ]:
def sample_instant_helper(df, scannum):
    errorflag = True
    scan_obj_list = []

In [ ]:
class series:
    def __init__(self, sample_list):
        return

In [8]:
directory = '/Users/anthonygironda/Documents/MDA_Scans_Seidler/'
meta = '7bmb1_10.19.22_EDXindex.csv'

sample_df = gen_spec_df(directory, meta)
sample_df.head()

sample_df
# for h in range(len(sample_df.index)):
#     positions = sample_df['Position'][h]
#     print(positions)
#     print('\n')


,ScanNum,FullName,pH,Days,Temperature,Atmosphere,SteelFiber,Aborted,Notes,Start (mm),End (mm),Step Size (mm),Slit Entrance (um),Scan Time,Rotation Speed (deg/s),FileName,Spectra,Position
0,1066,Foil Scan 1,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,NaN,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 221.0, 295.0, 314.0, 334.0, 354.0, 354....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1067,Foil Scan 2,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,NaN,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 203.0, 278.0, 317.0, 330.0, 369.0, 349....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1068,Foil Scan 3,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,NaN,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 849.0, 945.0, 1174.0, 1282.0, 1385.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1069,Mo W Energy Calibration,NaN,NaN,NaN,NaN,NaN,False,EDX Calibration,NaN,NaN,NaN,NaN,90.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 5366.0, 6316.0, 7253.0, 7844.0, 8473.0, ...",[]
4,1070,LaB6,NaN,NaN,NaN,NaN,NaN,False,EDX Commisioning,NaN,NaN,NaN,NaN,90.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 5138.0, 6247.0, 7093.0, 7730.0, 8211.0, ...",[]
5,1071,Calcite (CaCO3),NaN,NaN,NaN,NaN,NaN,False,EDX Calibration,NaN,NaN,NaN,NaN,90.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 5150.0, 6166.0, 7062.0, 7600.0, 8218.0, ...",[]
6,1072,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 1054.0, 1230.0, 1444.0, 1533.0, 1597.0, ...",[]
7,1073,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 106.0, 120.0, 128.0, 174.0, 164.0, 185....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,1074,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[[0.0, 98.0, 110.0, 135.0, 170.0, 159.0, 162.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,1075,High pH SF 25C 14d,High,14.0,25.0,100% CO2,True,False,Commisioning,13.0,6.0,1.00,250.0,20.0,NaN,/Users/anthonygironda/Documents/MDA_Scans_Seid...,"[0.0, 6217.0, 7384.0, 8390.0, 9519.0, 9998.0, ...",[]


In [ ]:
Mo_W_cal = scan_instant_helper(sample_df, 1069, 0, False)
calcite_ref = scan_instant_helper(sample_df, 1071, 0, True)

Mo_W_cal.spectra

# Mo_W_cal.quick_plot()
# Mo_W_cal.quick_scatter1()

In [ ]:
def df_to_objects(data_df):
    df_length = len(data_df.index)
    sample_array = []
    
    for a in range(df_length):
        sample_pH = None
        sample_steel = None
        sample_name = None
        sample_scans = []
        sample_file = None
        
#         if data_df['pH'][a] == 1:
#             sample_pH = 'High'
#         elif data_df['pH'][a] == 0:
#             sample_pH = 'Low'
            
#         if data_df['SteelFiber'][a] == 1:
#             sample_steel = 'Steel Fiber'
#         elif data_df['SteelFiber'][a] == 0:
#             sample_steel = 'No Steel Fiber'
            
        sample_name = data_df['FullName'][a]
        sample_file = data_df['FileName'][a]
        
        sample_scans.append(data_df['Position'][a])
        
        sample_array.append(sample(sample_name, sample_scans, sample_file))

    return sample_array

In [ ]:
samplefolder = "~/Documents/MDA_Scans_Seidler"


f = h5py.File(os.path.expanduser("~/Documents/MDA_Scans_Seidler/7bmb1_1076.hdf5"), 'r')
#list(f.keys())

list(f.attrs.keys())

#df = pd.read_csv("~/Documents/MDA_Scans_Seidler/7bmb1_10.19.22_EDXindex.csv")

#list(df.keys())
#df.head()

In [ ]:
print(dict(f.attrs))
